# 21-344 Final Project Code
# Juliette Wong (jnwong)

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_excel("Weather_PGH.xlsx")
df.tail()

,Date,Weather,High,Low,Precipitation (in),Wunderground?
726,2021-05-05,Cloudy,58,43,0.47,Yes
727,2021-05-06,Cloudy,57,37,0.05,Yes
728,2021-05-07,Scattered Showers,53,42,0.00,Yes
729,2021-05-08,Mostly Cloudy,50,42,0.18,Yes
730,2021-05-09,Scattered Showers,43,39,0.01,Yes


In [3]:
df.groupby("Wunderground?").count()

,Date,Weather,High,Low,Precipitation (in)
Wunderground?,,,,,
No,39,39,39,39,39
Yes,692,692,692,692,692


In [4]:
df.groupby("Weather").count()

,Date,High,Low,Precipitation (in),Wunderground?
Weather,,,,,
Cloudy,322,322,322,322,322
Foggy,8,8,8,8,8
Mostly Cloudy,119,119,119,119,119
Mostly Sunny,103,103,103,103,103
Partly Cloudy,71,71,71,71,71
Scattered Showers,55,55,55,55,55
Snow,49,49,49,49,49
Thunderstorm,4,4,4,4,4


## Converting Weather to an Integer Value

In [5]:
def weather_number(weather_name): 
    num = -1
    if weather_name == "Mostly Sunny": 
        num = 0
    elif weather_name == "Partly Cloudy":
        num = 1
    elif weather_name == "Mostly Cloudy":
        num = 2
    elif weather_name == "Cloudy": 
        num = 3
    elif weather_name == "Foggy":
        num = 4
    elif weather_name == "Scattered Showers":
        num = 5
    elif weather_name == "Thunderstorm": 
        num = 6
    elif weather_name == "Snow":
        num = 7
    return num

In [6]:
df["weather_num"] = df["Weather"].apply(weather_number)
df.head()

,Date,Weather,High,Low,Precipitation (in),Wunderground?,weather_num
0,2019-05-10,Cloudy,58,40,0.0,No,3
1,2019-05-11,Partly Cloudy,55,35,0.0,No,1
2,2019-05-12,Mostly Sunny,59,35,0.0,No,0
3,2019-05-13,Mostly Sunny,63,42,0.0,No,0
4,2019-05-14,Mostly Cloudy,64,43,0.0,No,2


## Creating 1-Step Transition Matrix

In [7]:
def create_transition_matrix(series): 
    counts = np.zeros((8, 8))
    n = len(series)
    for i in range(n-1):
        current = series[i]
        future = series[i+1]
        counts[current][future] += 1
    transition_matrix = counts/counts.sum(axis=1)[:,None]
    return counts, transition_matrix

In [23]:
count_mat, mat = create_transition_matrix(df["weather_num"])
mat
# Rounding does lead to some error rip
np.around(mat, decimals=4)
# np.around(mat, decimals=4).sum(axis=1)[:, None] 

array([[0.3204, 0.1553, 0.1748, 0.3301, 0.    , 0.    , 0.0097, 0.0097],
       [0.1831, 0.1408, 0.2113, 0.3521, 0.0141, 0.0986, 0.    , 0.    ],
       [0.1513, 0.1513, 0.2773, 0.3277, 0.    , 0.0504, 0.0252, 0.0168],
       [0.0714, 0.0652, 0.1273, 0.5373, 0.0155, 0.1056, 0.    , 0.0776],
       [0.25  , 0.    , 0.25  , 0.375 , 0.125 , 0.    , 0.    , 0.    ],
       [0.1481, 0.0741, 0.0926, 0.4444, 0.0185, 0.1481, 0.    , 0.0741],
       [0.5   , 0.    , 0.25  , 0.25  , 0.    , 0.    , 0.    , 0.    ],
       [0.0816, 0.0408, 0.0816, 0.449 , 0.    , 0.    , 0.    , 0.3469]])

## Creating Multi-Step Transition Matrix

In [26]:
np.around(np.linalg.matrix_power(mat, 2), 4)

array([[0.1868, 0.12  , 0.1825, 0.4019, 0.0073, 0.059 , 0.0075, 0.035 ],
       [0.1597, 0.1105, 0.1778, 0.4176, 0.011 , 0.0763, 0.0071, 0.04  ],
       [0.163 , 0.1125, 0.1894, 0.4064, 0.0082, 0.071 , 0.0085, 0.0411],
       [0.1183, 0.0856, 0.15  , 0.4645, 0.0132, 0.0852, 0.0039, 0.0793],
       [0.1759, 0.1011, 0.192 , 0.4128, 0.0214, 0.0522, 0.0087, 0.0357],
       [0.1394, 0.0904, 0.1482, 0.4502, 0.013 , 0.0808, 0.0038, 0.0742],
       [0.2159, 0.1318, 0.1885, 0.3813, 0.0039, 0.039 , 0.0112, 0.0285],
       [0.1064, 0.0742, 0.131 , 0.4651, 0.0075, 0.0555, 0.0029, 0.1574]])

In [29]:
np.around(np.linalg.matrix_power(mat, 7), 4)

array([[0.1415, 0.0974, 0.1632, 0.4393, 0.011 , 0.0754, 0.0055, 0.0667],
       [0.1414, 0.0974, 0.1631, 0.4394, 0.011 , 0.0754, 0.0055, 0.0669],
       [0.1414, 0.0974, 0.1631, 0.4394, 0.011 , 0.0754, 0.0055, 0.0668],
       [0.1411, 0.0972, 0.1629, 0.4397, 0.011 , 0.0754, 0.0055, 0.0673],
       [0.1415, 0.0974, 0.1632, 0.4393, 0.011 , 0.0754, 0.0055, 0.0668],
       [0.1412, 0.0972, 0.1629, 0.4396, 0.011 , 0.0754, 0.0055, 0.0672],
       [0.1416, 0.0975, 0.1632, 0.4392, 0.0109, 0.0754, 0.0055, 0.0666],
       [0.1408, 0.097 , 0.1627, 0.4399, 0.011 , 0.0754, 0.0055, 0.0677]])

In [30]:
np.around(np.linalg.matrix_power(mat, 7), 4).sum(axis=1)[:, None] 

array([[1.    ],
       [1.0001],
       [1.    ],
       [1.0001],
       [1.0001],
       [1.    ],
       [0.9999],
       [1.    ]])

## Limiting Distribution

In [11]:
P_thousand = np.linalg.matrix_power(mat, 1000)

In [39]:
pi = P_thousand[0]
pi

array([0.14124014, 0.09728391, 0.16296765, 0.43957079, 0.01096225,
       0.07539167, 0.00547969, 0.06710389])

In [42]:
np.around(pi, 4)

array([0.1412, 0.0973, 0.163 , 0.4396, 0.011 , 0.0754, 0.0055, 0.0671])

## Showing why Markov Chains work

### Independence Model does not Hold

In [13]:
weather_counts = df.groupby("weather_num").count()["Date"]
weather_prop = weather_counts / weather_counts.sum()

In [14]:
expected = np.zeros((8, 8))
for i in range(8):
    for j in range(8): 
        expected[i][j] = weather_prop[i]*weather_prop[j]*731
np.around(expected)

array([[ 15.,  10.,  17.,  45.,   1.,   8.,   1.,   7.],
       [ 10.,   7.,  12.,  31.,   1.,   5.,   0.,   5.],
       [ 17.,  12.,  19.,  52.,   1.,   9.,   1.,   8.],
       [ 45.,  31.,  52., 142.,   4.,  24.,   2.,  22.],
       [  1.,   1.,   1.,   4.,   0.,   1.,   0.,   1.],
       [  8.,   5.,   9.,  24.,   1.,   4.,   0.,   4.],
       [  1.,   0.,   1.,   2.,   0.,   0.,   0.,   0.],
       [  7.,   5.,   8.,  22.,   1.,   4.,   0.,   3.]])

In [15]:
count_mat

array([[ 33.,  16.,  18.,  34.,   0.,   0.,   1.,   1.],
       [ 13.,  10.,  15.,  25.,   1.,   7.,   0.,   0.],
       [ 18.,  18.,  33.,  39.,   0.,   6.,   3.,   2.],
       [ 23.,  21.,  41., 173.,   5.,  34.,   0.,  25.],
       [  2.,   0.,   2.,   3.,   1.,   0.,   0.,   0.],
       [  8.,   4.,   5.,  24.,   1.,   8.,   0.,   4.],
       [  2.,   0.,   1.,   1.,   0.,   0.,   0.,   0.],
       [  4.,   2.,   4.,  22.,   0.,   0.,   0.,  17.]])

### 1st-Order Markov Chain

In [16]:
def create_counts(series): 
    counts = np.zeros((64, 8))
    n = len(series)
    for i in range(n-2):
        current = series[i]
        next_val = series[i+1]
        future_val = series[i+2]
        counts[8*current + next_val][future_val] += 1
    return counts
actual = create_counts(df["weather_num"])
actual

array([[ 11.,   5.,   4.,  13.,   0.,   0.,   0.,   0.],
       [  2.,   3.,   1.,   7.,   1.,   2.,   0.,   0.],
       [  2.,   0.,   5.,   9.,   0.,   1.,   1.,   0.],
       [  2.,   3.,   8.,  15.,   1.,   4.,   0.,   1.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  6.,   0.,   5.,   2.,   0.,   0.,   0.,   0.],
       [  3.,   1.,   2.,   2.,   0.,   2.,   0.,   0.],
       [  2.,   3.,   4.,   5.,   0.,   0.,   0.,   1.],
       [  4.,   2.,   7.,   8.,   0.,   4.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.],
       [  1.,   0.,   1.,   3.,   0.,   1.,   0.,   1.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
       [  4.,   4.,   5.,   5.,   0.,   0.,   0.,   0.],
       [  4.,   4.,   3.,   6.,

In [17]:
actual_row_sum = np.sum(actual, axis = 1)
expected = np.zeros((64, 8))

for i in range(64):
    for j in range(8):
        expected[i][j] = mat[i % 8][j] * actual_row_sum[i]
np.around(expected)

array([[11.,  5.,  6., 11.,  0.,  0.,  0.,  0.],
       [ 3.,  2.,  3.,  6.,  0.,  2.,  0.,  0.],
       [ 3.,  3.,  5.,  6.,  0.,  1.,  0.,  0.],
       [ 2.,  2.,  4., 18.,  1.,  4.,  0.,  3.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 4.,  2.,  2.,  4.,  0.,  0.,  0.,  0.],
       [ 2.,  1.,  2.,  4.,  0.,  1.,  0.,  0.],
       [ 2.,  2.,  4.,  5.,  0.,  1.,  0.,  0.],
       [ 2.,  2.,  3., 13.,  0.,  3.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  3.,  0.,  1.,  0.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 6.,  3.,  3.,  6.,  0.,  0.,  0.,  0.],
       [ 3.,  3.,  4.,  6.,  0.,  2.,  0.,  0.],
       [ 5.,  5.,  9., 11.,  0.,  2.,  1.,  1.],
       [ 3.,  3.,  5., 21.,  1.,  4.,  0.,  3.],
       [ 0.,  0.,  0